In [1]:
import numpy as np
import pandas as pd
import spacy
import matplotlib.pyplot as plt 
import seaborn as sns
import nltk

In [2]:
textfiles = "C:\\workspace\\studyvideo\\Natural Language Processing with Python\\1. Introduction\\5.1 UPDATED_NLP_COURSE.zip\\UPDATED_NLP_COURSE\\TextFiles"

In [3]:
import os
movie_rw = os.path.join(textfiles,'moviereviews.tsv')
mr_data=pd.read_csv(movie_rw,sep="\t")

In [4]:
mr_data.head()

,label,review
0,neg,how do films like mouse hunt get into theatres...
1,neg,some talented actresses are blessed with a dem...
2,pos,this has been an extraordinary year for austra...
3,pos,according to hollywood movies made in last few...
4,neg,my first press screening of 1998 and already i...


In [5]:
len(mr_data)

2000

In [7]:
from IPython.display import Markdown, display
display(Markdown('> '+mr_data['review'][0]))

> how do films like mouse hunt get into theatres ? 
isn't there a law or something ? 
this diabolical load of claptrap from steven speilberg's dreamworks studio is hollywood family fare at its deadly worst . 
mouse hunt takes the bare threads of a plot and tries to prop it up with overacting and flat-out stupid slapstick that makes comedies like jingle all the way look decent by comparison . 
writer adam rifkin and director gore verbinski are the names chiefly responsible for this swill . 
the plot , for what its worth , concerns two brothers ( nathan lane and an appalling lee evens ) who inherit a poorly run string factory and a seemingly worthless house from their eccentric father . 
deciding to check out the long-abandoned house , they soon learn that it's worth a fortune and set about selling it in auction to the highest bidder . 
but battling them at every turn is a very smart mouse , happy with his run-down little abode and wanting it to stay that way . 
the story alternates between unfunny scenes of the brothers bickering over what to do with their inheritance and endless action sequences as the two take on their increasingly determined furry foe . 
whatever promise the film starts with soon deteriorates into boring dialogue , terrible overacting , and increasingly uninspired slapstick that becomes all sound and fury , signifying nothing . 
the script becomes so unspeakably bad that the best line poor lee evens can utter after another run in with the rodent is : " i hate that mouse " . 
oh cringe ! 
this is home alone all over again , and ten times worse . 
one touching scene early on is worth mentioning . 
we follow the mouse through a maze of walls and pipes until he arrives at his makeshift abode somewhere in a wall . 
he jumps into a tiny bed , pulls up a makeshift sheet and snuggles up to sleep , seemingly happy and just wanting to be left alone . 
it's a magical little moment in an otherwise soulless film . 
a message to speilberg : if you want dreamworks to be associated with some kind of artistic credibility , then either give all concerned in mouse hunt a swift kick up the arse or hire yourself some decent writers and directors . 
this kind of rubbish will just not do at all . 


In [9]:
mr_data.isnull().sum()

label      0
review    35
dtype: int64

In [10]:
mr_data.dropna(inplace=True)

In [11]:
mr_data.isnull().sum()

label     0
review    0
dtype: int64

In [12]:
len(mr_data)

1965

In [17]:
for i,j,k in list(mr_data.itertuples()):
    print(i,"---",j,"---",k[:50])
    break

0 --- neg --- how do films like mouse hunt get into theatres ? 


In [19]:
blanks = []  # start with an empty list

for i,lb,rv in mr_data.itertuples():  # iterate over the DataFrame
    if type(rv)==str:            # avoid NaN values
        if rv.isspace():         # test 'review' for whitespace
            blanks.append(i)     # add matching index numbers to the list
        
print(len(blanks), 'blanks: ', blanks)

27 blanks:  [57, 71, 147, 151, 283, 307, 313, 323, 343, 351, 427, 501, 633, 675, 815, 851, 977, 1079, 1299, 1455, 1493, 1525, 1531, 1763, 1851, 1905, 1993]


In [20]:
mr_data.drop(blanks, inplace=True)

len(mr_data)

1938

In [21]:
mr_data['label'].value_counts()

neg    969
pos    969
Name: label, dtype: int64

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [23]:
X=mr_data['review']
y=mr_data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [25]:
text_clf_mnb = Pipeline(
    [('tfidf', TfidfVectorizer()),
     ('clf', MultinomialNB())
    ]
)

text_clf_lsvc = Pipeline(
    [('tfidf', TfidfVectorizer()),
     ('clf', LinearSVC())
    ]
)


In [26]:
text_clf_mnb.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...rue,
        vocabulary=None)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [27]:
mnb_pred = text_clf_mnb.predict(X_test)

In [29]:
def pred_report(y_test,y_pred):
    print ("""
    Classification Report :\n {0}
    Confusion Matrix      :\n {1}
    Accuracy              : {2}
    """ .format(classification_report(y_test,y_pred),confusion_matrix(y_test,y_pred),accuracy_score(y_test,y_pred)))

In [30]:
pred_report(y_test,mnb_pred)


    Classification Report :
               precision    recall  f1-score   support

         neg       0.69      0.93      0.79       308
         pos       0.91      0.61      0.73       332

   micro avg       0.76      0.76      0.76       640
   macro avg       0.80      0.77      0.76       640
weighted avg       0.80      0.76      0.76       640

    Confusion Matrix      :
 [[287  21]
 [130 202]]
    Accuracy              : 0.7640625
    


In [31]:
text_clf_lsvc.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [32]:
lsvc_pred = text_clf_lsvc.predict(X_test)

In [33]:
pred_report(y_test,lsvc_pred)


    Classification Report :
               precision    recall  f1-score   support

         neg       0.84      0.84      0.84       308
         pos       0.85      0.85      0.85       332

   micro avg       0.85      0.85      0.85       640
   macro avg       0.85      0.85      0.85       640
weighted avg       0.85      0.85      0.85       640

    Confusion Matrix      :
 [[259  49]
 [ 49 283]]
    Accuracy              : 0.846875
    


### Check with stop words

In [34]:
stopwords = ['a', 'about', 'an', 'and', 'are', 'as', 'at', 'be', 'been', 'but', 'by', 'can', \
             'even', 'ever', 'for', 'from', 'get', 'had', 'has', 'have', 'he', 'her', 'hers', 'his', \
             'how', 'i', 'if', 'in', 'into', 'is', 'it', 'its', 'just', 'me', 'my', 'of', 'on', 'or', \
             'see', 'seen', 'she', 'so', 'than', 'that', 'the', 'their', 'there', 'they', 'this', \
             'to', 'was', 'we', 'were', 'what', 'when', 'which', 'who', 'will', 'with', 'you']

In [35]:
text_clf_lsvc_sw = Pipeline(
    [('tfidf', TfidfVectorizer(stop_words=stopwords)),
     ('clf', LinearSVC())
    ]
)


In [36]:
text_clf_lsvc_sw.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [37]:
lsvc_sw_pred = text_clf_lsvc_sw.predict(X_test)
pred_report(y_test,lsvc_sw_pred)


    Classification Report :
               precision    recall  f1-score   support

         neg       0.84      0.83      0.84       308
         pos       0.85      0.86      0.85       332

   micro avg       0.84      0.84      0.84       640
   macro avg       0.84      0.84      0.84       640
weighted avg       0.84      0.84      0.84       640

    Confusion Matrix      :
 [[256  52]
 [ 48 284]]
    Accuracy              : 0.84375
    


In [40]:
movie_reviews = "This movie indeed sucks!!! no other words!!!"
print("Naive Bayes               : ",text_clf_mnb.predict([movie_reviews]))
print("Linear SVM                : ",text_clf_lsvc.predict([movie_reviews]))
print("Linear SVM with Stopwords : ",text_clf_lsvc_sw.predict([movie_reviews]))

Naive Bayes               :  ['neg']
Linear SVM                :  ['neg']
Linear SVM with Stopwords :  ['neg']


In [41]:
movie_reviews = "Love It, simply awesome... wowieee!!!"
print("Naive Bayes               : ",text_clf_mnb.predict([movie_reviews]))
print("Linear SVM                : ",text_clf_lsvc.predict([movie_reviews]))
print("Linear SVM with Stopwords : ",text_clf_lsvc_sw.predict([movie_reviews]))

Naive Bayes               :  ['pos']
Linear SVM                :  ['neg']
Linear SVM with Stopwords :  ['pos']


In [42]:
negative_review = "Amazon offers a credit card through Chase bank, while it may sound good, especially during the Christmas season BEWARE! Amazon CHANGES your payment method WITHOUT NOTIFYING YOU. If you set up your payments to be made through your bank, debit card, you may not notice it right away. Suddenly you have a credit card bill you weren't expecting. Amazon DID NOT have the authorization to make this change, but did so anyway. This is the second time this has happened!"
print("Naive Bayes               : ",text_clf_mnb.predict([negative_review]))
print("Linear SVM                : ",text_clf_lsvc.predict([negative_review]))
print("Linear SVM with Stopwords : ",text_clf_lsvc_sw.predict([negative_review]))

Naive Bayes               :  ['neg']
Linear SVM                :  ['neg']
Linear SVM with Stopwords :  ['neg']


In [43]:
comment = "Reminder to complete pulse timesheet routing by 12pm today"
print("Naive Bayes               : ",text_clf_mnb.predict([comment]))
print("Linear SVM                : ",text_clf_lsvc.predict([comment]))
print("Linear SVM with Stopwords : ",text_clf_lsvc_sw.predict([comment]))

Naive Bayes               :  ['pos']
Linear SVM                :  ['neg']
Linear SVM with Stopwords :  ['pos']


In [44]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ashok\AppData\Roaming\nltk_data...


True

In [46]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
SIA=SentimentIntensityAnalyzer()

In [48]:
Amazon_rw1 = "They sell falsely rated items. Purchases are made based on reviews but they are not true reviews. Thy sell cheap items and they do no longer have an easy return policy or take responsibility for the items sold."

In [49]:
SIA.polarity_scores(Amazon_rw1)

{'neg': 0.13, 'neu': 0.784, 'pos': 0.086, 'compound': -0.2378}

In [50]:
### Amazon Movie Review

In [57]:
amazon_review_file=os.path.join(textfiles,'amazonreviews.tsv')

In [58]:
amzn_data  = pd.read_csv(amazon_review_file,sep="\t")

In [59]:
amzn_data.head()

,label,review
0,pos,Stuning even for the non-gamer: This sound tra...
1,pos,The best soundtrack ever to anything.: I'm rea...
2,pos,Amazing!: This soundtrack is my favorite music...
3,pos,Excellent Soundtrack: I truly like this soundt...
4,pos,"Remember, Pull Your Jaw Off The Floor After He..."


In [61]:
amzn_data.isnull().sum()

label     0
review    0
dtype: int64

In [62]:
from IPython.display import Markdown, display
display(Markdown('> '+amzn_data['review'][0]))

> Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^

In [65]:
amzn_data['review'].apply(lambda review : SIA.polarity_scores(review)).head()

0    {'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'co...
1    {'neg': 0.018, 'neu': 0.837, 'pos': 0.145, 'co...
2    {'neg': 0.04, 'neu': 0.692, 'pos': 0.268, 'com...
3    {'neg': 0.09, 'neu': 0.615, 'pos': 0.295, 'com...
4    {'neg': 0.0, 'neu': 0.746, 'pos': 0.254, 'comp...
Name: review, dtype: object

In [66]:
amzn_data['scores'] = amzn_data['review'].apply(lambda review : SIA.polarity_scores(review))

In [67]:
amzn_data.head()

,label,review,scores
0,pos,Stuning even for the non-gamer: This sound tra...,"{'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'co..."
1,pos,The best soundtrack ever to anything.: I'm rea...,"{'neg': 0.018, 'neu': 0.837, 'pos': 0.145, 'co..."
2,pos,Amazing!: This soundtrack is my favorite music...,"{'neg': 0.04, 'neu': 0.692, 'pos': 0.268, 'com..."
3,pos,Excellent Soundtrack: I truly like this soundt...,"{'neg': 0.09, 'neu': 0.615, 'pos': 0.295, 'com..."
4,pos,"Remember, Pull Your Jaw Off The Floor After He...","{'neg': 0.0, 'neu': 0.746, 'pos': 0.254, 'comp..."


In [70]:
amzn_data['compound']=amzn_data['scores'].apply(lambda comp_score: comp_score['compound'])

In [71]:
amzn_data.head()

,label,review,scores,compound
0,pos,Stuning even for the non-gamer: This sound tra...,"{'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'co...",0.9454
1,pos,The best soundtrack ever to anything.: I'm rea...,"{'neg': 0.018, 'neu': 0.837, 'pos': 0.145, 'co...",0.8957
2,pos,Amazing!: This soundtrack is my favorite music...,"{'neg': 0.04, 'neu': 0.692, 'pos': 0.268, 'com...",0.9858
3,pos,Excellent Soundtrack: I truly like this soundt...,"{'neg': 0.09, 'neu': 0.615, 'pos': 0.295, 'com...",0.9814
4,pos,"Remember, Pull Your Jaw Off The Floor After He...","{'neg': 0.0, 'neu': 0.746, 'pos': 0.254, 'comp...",0.9781


In [72]:
amzn_data['comp_score'] = amzn_data['compound'].apply(lambda score: 'pos' if score >=0 else 'neg')

In [73]:
amzn_data.head()

,label,review,scores,compound,comp_score
0,pos,Stuning even for the non-gamer: This sound tra...,"{'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'co...",0.9454,pos
1,pos,The best soundtrack ever to anything.: I'm rea...,"{'neg': 0.018, 'neu': 0.837, 'pos': 0.145, 'co...",0.8957,pos
2,pos,Amazing!: This soundtrack is my favorite music...,"{'neg': 0.04, 'neu': 0.692, 'pos': 0.268, 'com...",0.9858,pos
3,pos,Excellent Soundtrack: I truly like this soundt...,"{'neg': 0.09, 'neu': 0.615, 'pos': 0.295, 'com...",0.9814,pos
4,pos,"Remember, Pull Your Jaw Off The Floor After He...","{'neg': 0.0, 'neu': 0.746, 'pos': 0.254, 'comp...",0.9781,pos


In [74]:
pred_report(amzn_data['label'],amzn_data['comp_score'])


    Classification Report :
               precision    recall  f1-score   support

         neg       0.86      0.51      0.64      5097
         pos       0.64      0.91      0.75      4903

   micro avg       0.71      0.71      0.71     10000
   macro avg       0.75      0.71      0.70     10000
weighted avg       0.75      0.71      0.70     10000

    Confusion Matrix      :
 [[2623 2474]
 [ 435 4468]]
    Accuracy              : 0.7091
    


In [75]:
### NMF

In [78]:
os.listdir(textfiles)

['amazonreviews.tsv',
 'elon_tweets.txt',
 'elon_tweets2.txt',
 'huckfinn.txt',
 'moviereviews.tsv',
 'moviereviews2.tsv',
 'owlcreek.txt',
 'peterrabbit.txt',
 'reaganomics.txt',
 'reuters.csv',
 'smsspamcollection.tsv',
 'sms_readme.txt']

In [80]:
reuters_data=pd.read_csv(os.path.join(textfiles,'reuters.csv'))

In [81]:
reuters_data.head()

,headline_text,category
0,Southern European bond yields hit multi-week lows,3
1,BRIEF-LG sells its entire stake in unit LG Lif...,6
2,BRIEF-Golden Wheel Tiandi says unit confirms s...,3
3,BRIEF-Sunshine 100 China Holdings Dec contract...,3
4,Euro zone stocks start 2017 with new one-year ...,3


In [82]:
tfidf = TfidfVectorizer(max_df=0.9, min_df=2, stop_words='english')

In [83]:
reuter_dm = tfidf.fit_transform(reuters_data['headline_text'])

In [84]:
reuter_dm

<999x1183 sparse matrix of type '<class 'numpy.float64'>'
	with 6251 stored elements in Compressed Sparse Row format>

In [85]:
from sklearn.decomposition import NMF

In [140]:
nmf_model = NMF(n_components=8,random_state=42)

In [141]:
nmf_model.fit(reuter_dm)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=8, random_state=42, shuffle=False, solver='cd', tol=0.0001,
  verbose=0)

In [143]:
for index,topic in enumerate(nmf_model.components_):
    print(f'THE TOP 8 WORDS FOR TOPIC #{index}')
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 8 WORDS FOR TOPIC #0
['352', 'ftws', '524', 'cmd_data_check_uint16', '3087', '1426', '350', '2126', '407', 'dev', '211', '142', '127', '1390', '19']


THE TOP 8 WORDS FOR TOPIC #1
['222', 'address', 'gengtype', 'database', '429', '290', 'bytes', '1970', '292', '160', '19', 'dummy', '104', 'control', 'atf_addac']


THE TOP 8 WORDS FOR TOPIC #2
['ar9300device', 'fieldread', '43', '222', 'format', 'dup', 'boot', '310', '994', 'fqdn', '1043', 'getgid03', '399', '725', 'elmtlen2']


THE TOP 8 WORDS FOR TOPIC #3
['183', '957', 'generate_translations', '279', 'comment', '15', '725', 'defaults', 'configsets', 'expr', 'cmd_strtol', '1793', '2099', 'clnt_tcp', '627']


THE TOP 8 WORDS FOR TOPIC #4
['1468', '474', '937', 'build_dir', 'bindresvport', 'dev_read_data', '589', 'ar9300device', '144', 'chroma_elim_threshold', '408', 'counter', 'fatal', 'btmode', 'break']


THE TOP 8 WORDS FOR TOPIC #5
['917', 'cmd_data_check_uint16', '957', 'gfc0326e', 'definition', 'ezsp', '1948', '667', 'dumm

In [144]:
reuter_topic_results = nmf_model.transform(reuter_dm)

In [145]:
reuter_topic_results.argmax(axis=1)

array([7, 5, 6, 1, 7, 2, 0, 5, 7, 6, 7, 7, 1, 1, 7, 4, 7, 1, 1, 7, 2, 5,
       6, 2, 1, 1, 7, 0, 6, 1, 0, 0, 6, 6, 7, 6, 0, 5, 0, 7, 6, 1, 2, 2,
       5, 0, 5, 6, 0, 5, 3, 6, 3, 1, 1, 5, 3, 6, 6, 2, 6, 0, 6, 0, 1, 7,
       3, 6, 6, 7, 1, 0, 2, 0, 7, 0, 7, 6, 7, 7, 6, 0, 5, 0, 7, 5, 3, 1,
       0, 3, 2, 7, 1, 3, 5, 6, 0, 7, 0, 1, 7, 7, 6, 1, 0, 5, 0, 3, 5, 3,
       3, 0, 7, 6, 1, 3, 3, 3, 1, 1, 6, 5, 1, 2, 3, 5, 7, 0, 3, 1, 7, 3,
       6, 6, 6, 0, 0, 2, 3, 6, 2, 1, 3, 1, 3, 6, 1, 3, 0, 3, 3, 6, 3, 7,
       1, 1, 7, 3, 7, 7, 3, 0, 6, 2, 3, 5, 7, 3, 7, 6, 6, 7, 3, 5, 7, 0,
       7, 5, 7, 6, 3, 7, 3, 5, 7, 0, 6, 0, 0, 7, 7, 3, 6, 1, 6, 6, 5, 5,
       0, 5, 5, 0, 5, 7, 7, 7, 3, 3, 1, 5, 3, 1, 0, 1, 6, 7, 1, 2, 2, 0,
       5, 7, 6, 7, 6, 5, 1, 6, 6, 7, 7, 5, 0, 0, 1, 1, 7, 6, 0, 6, 7, 2,
       2, 7, 7, 5, 5, 0, 2, 6, 3, 0, 3, 6, 7, 7, 5, 5, 0, 6, 7, 3, 1, 6,
       3, 3, 6, 7, 7, 6, 6, 6, 5, 3, 2, 1, 2, 1, 1, 3, 1, 3, 1, 1, 0, 7,
       7, 5, 0, 7, 7, 5, 5, 5, 1, 6, 0, 0, 5, 5, 5,

In [146]:
reuters_data['cat_pred'] = reuter_topic_results.argmax(axis=1)
reuters_data.head()

,headline_text,category,cat_pred
0,Southern European bond yields hit multi-week lows,3,7
1,BRIEF-LG sells its entire stake in unit LG Lif...,6,5
2,BRIEF-Golden Wheel Tiandi says unit confirms s...,3,6
3,BRIEF-Sunshine 100 China Holdings Dec contract...,3,1
4,Euro zone stocks start 2017 with new one-year ...,3,7


In [147]:
pred_report(reuters_data['category'],reuters_data['cat_pred'] )


    Classification Report :
               precision    recall  f1-score   support

           0       0.01      0.07      0.02        30
           1       0.08      0.13      0.10        61
           2       0.00      0.00      0.00         7
           3       0.64      0.11      0.19       584
           4       0.11      0.01      0.01       188
           5       0.06      0.11      0.08        64
           6       0.05      0.14      0.08        64
           7       0.00      0.00      0.00         1

   micro avg       0.09      0.09      0.09       999
   macro avg       0.12      0.07      0.06       999
weighted avg       0.41      0.09      0.13       999

    Confusion Matrix      :
 [[  2   1   2   1   0   0  14  10]
 [  5   8   5   4   8   3  11  17]
 [  0   1   0   2   0   0   0   4]
 [113  77  65  65   0  80  85  99]
 [ 20  12   9  24   1   8  45  69]
 [ 32   4   0   3   0   7  10   8]
 [ 34   3   3   2   0  11   9   2]
 [  1   0   0   0   0   0   0   0]]
    Accur

In [104]:
reuters_data.drop('cat_pred',axis=1,inplace=True)

In [113]:
reuters_data['category'].value_counts(sort=False)

0     30
1     61
2      7
3    584
4    188
5     64
6     64
7      1
Name: category, dtype: int64

In [116]:
reuters_data['category'].unique()

array([3, 6, 7, 4, 1, 5, 2, 0], dtype=int64)

In [117]:
download_folder = "C:\\Users\\ashok\\Downloads"

In [119]:
logfile = os.path.join(download_folder,'All_wErrorRemoved_Feb.log')

In [120]:
with open(logfile) as f:
    rows=f.readlines()
rows[:2]

['nohup: ignoring input\n', "Checking 'uuencode'... ok.\n"]

In [121]:
rowss = [ row for row in rows if "warning:" in row ]

In [122]:
rowss[3:]

["gbacktrace.c:302:22: warning: ignoring return value of 'dup', declared with attribute warn_unused_result [-Wunused-result]\n",
 "gbacktrace.c:303:22: warning: ignoring return value of 'dup', declared with attribute warn_unused_result [-Wunused-result]\n",
 "gbacktrace.c:318:9: warning: ignoring return value of 'write', declared with attribute warn_unused_result [-Wunused-result]\n",
 "gbacktrace.c:319:9: warning: ignoring return value of 'write', declared with attribute warn_unused_result [-Wunused-result]\n",
 "gbacktrace.c:320:9: warning: ignoring return value of 'write', declared with attribute warn_unused_result [-Wunused-result]\n",
 'glibintl.h:37:42: warning: statement with no effect [-Wunused-value]\n',
 "install-info: warning: no info dir entry in `/local/mnt/workspace/ashok/wall_Error_repo_Feb14/date_4_4/qsdk/staging_dir/host/share/info/automake-history.info'\n",
 "Makefile.am:113: warning: source file 'loaders/dld_link.c' is in a subdirectory,\n",
 'automake: warning: poss

In [123]:
compiler = tfidf.fit_transform(rowss[3:])

In [124]:
compiler_model = NMF(n_components=20)

In [125]:
compiler_model.fit(compiler)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=20, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [127]:
for index,topic in enumerate(compiler_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['date_4_4', 'wall_error_repo_feb14', 'ashok', 'arm_cortex', '16_eabi', '0_musl', 'a7_gcc', 'toolchain', 'staging_dir', 'wcpp', 'redirecting', 'incorrect', 'include', 'sys', 'fcntl']


THE TOP 15 WORDS FOR TOPIC #1
['33', '77', '80211stats', 'u_int64_t', 'src', 'sstrip', 'aka', 'format', 'wformat', 'unsigned', 'expects', 'type', 'argument', 'long', 'int']


THE TOP 15 WORDS FOR TOPIC #2
['get80211param', '178', 'vorbis_ftoi', 'used', '13', 'wunused', '12', 'linux', 'function', 'declared', 'tools', 'wlanconfig', 'static', 'os', 'defined']


THE TOP 15 WORDS FOR TOPIC #3
['nat46', 'iso', 'macro', 'whitespace', 'c99', 'requires', 'redefined', 'iface_mgr_cfg_support', 'cc1', 'option', 'objc', 'nostdinc', 'valid', 'line', 'command']


THE TOP 15 WORDS FOR TOPIC #4
['2017', '09', 'utils', 'ltp', 'func_tests', 'sctp', 'testlib', 'sctputil', 'unsigned', 'integer', 'comparison', 'compare', 'signed', 'expressions', 'wsign']


THE TOP 15 WORDS FOR TOPIC #5
['wireless

In [128]:
compiler_model_results = compiler_model.transform(compiler)

In [133]:
set(compiler_model_results.argmax(axis=1))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}